# This will be a simple notebook to try and challenge the model created in 
#### https://github.com/acantril/learn-cantrill-io-labs/tree/master/aws-pet-rekognition-ecr
##### Description to be update
* Ample documentation and comparisons will be on Github in later stages

In [2]:
# For lib/pkg installs
#!py -m pip install zipfile39

###### Imports and prep

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.optimizers import RMSprop

In [2]:
base_dir = 'C:/Users/gamze/Desktop/Study/ML/Projects/Datasets/Cats_Dogs_Class_Reko/acantrill' # Local data set Train:160__Val:80
periph_dir = 'C:/Users/gamze/Desktop/Study/ML/Projects/Datasets/Cats_Dogs_Class_Reko/google_api' # google API dataset Train:1000___Val:500
train_dir = os.path.join(base_dir,'train')
validation_dir = os.path.join(base_dir,'validation')

In [3]:
train_cats_dir = os.path.join(train_dir, 'cats') # 1000 samples --> 160 
train_dogs_dir = os.path.join(train_dir, 'dogs') # 1000 samples --> 160

In [4]:
validation_cats_dir = os.path.join(validation_dir, 'cats') # 500 samples --> 80
validation_dogs_dir = os.path.join(validation_dir, 'dogs') # 500 samples --> 80

### First Pre-trained model to implement is the: 
## Inception V3


Image augmentation since the dataset used here is lacking in size


In [6]:
train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range=40,
                                   width_shift_range=.2, height_shift_range=.2,
                                   shear_range=.2, zoom_range=.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
train_generator = train_datagen.flow_from_directory(train_dir, batch_size=20, 
                                                    class_mode='binary', target_size=(150,150))
validation_generator = test_datagen.flow_from_directory(validation_dir, batch_size=20, 
                                                    class_mode='binary', target_size=(150,150))

Found 320 images belonging to 2 classes.
Found 162 images belonging to 2 classes.


* Load Base Model of the Inception V3


In [9]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

base_model = InceptionV3(input_shape=(150,150,3), 
                   include_top = False, # leave out laft FC layer
                   weights='imagenet')


87910968/87910968 [==============================] - 60s 1us/step


* No need to train all layers

In [10]:
for layer in base_model.layers:
    layer.trainable = False

###### Compile with normal settings
* Fine Tuning later

In [12]:
# Flatten to 1 dim
x = Flatten()(base_model.output)

# FC layer with the 1024 hidden units and a ReLU 
x = Dense(1024, activation='relu')(x)

# Setting the dropout for .5 for now as def, but will decrease later if no hyper parameter tuning is done
x = Dropout(0.5)(x)

# Sigmoid for output
x = layers.Dense(1,activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input,x)
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=.0001), 
              loss='binary_crossentropy',metrics=['acc'])


In [15]:
inchistory = model.fit(train_generator, 
                    validation_data=validation_generator,
                    steps_per_epoch=16, epochs=10)

Epoch 1/10
16/16 [==============================] - 48s 3s/step - loss: 0.9862 - acc: 0.8469 - val_loss: 0.5924 - val_acc: 0.8827
Epoch 2/10
16/16 [==============================] - 57s 4s/step - loss: 0.5805 - acc: 0.8625 - val_loss: 0.1918 - val_acc: 0.9506
Epoch 3/10
16/16 [==============================] - 58s 4s/step - loss: 0.4961 - acc: 0.8844 - val_loss: 0.1017 - val_acc: 0.9630
Epoch 4/10
16/16 [==============================] - 47s 3s/step - loss: 0.5332 - acc: 0.8750 - val_loss: 0.1896 - val_acc: 0.9506
Epoch 5/10
16/16 [==============================] - 53s 3s/step - loss: 0.5313 - acc: 0.8844 - val_loss: 0.1925 - val_acc: 0.9444
Epoch 6/10
16/16 [==============================] - 102s 6s/step - loss: 0.4884 - acc: 0.8781 - val_loss: 0.7705 - val_acc: 0.8827
Epoch 7/10
16/16 [==============================] - 58s 3s/step - loss: 0.5984 - acc: 0.8750 - val_loss: 0.1979 - val_acc: 0.9506
Epoch 8/10
16/16 [==============================] - 50s 3s/step - loss: 0.4730 - acc: 0.8